In [2]:
import os
import os.path as op
if op.exists("data") == False:
    os.mkdir("data")

In [14]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_reads/phages.txt data/

phages.txt                                    100% 2583     2.5KB/s   00:00    


In [ ]:
with open(phagelist) as ih, open("multicontig.txt", "w") as oh1, open("asm_contig_cts.txt", "w") as oh2:
    print("phage", "assembly1_contig_count", sep="\t", file=oh2)
    for p in phagelist:
        asm1 = "/nobackup1/jbrown/nahant_phage_reads/big_pipeline/{p}step1.contig1.fasta"
        contigs = 0
        with open(asm1) as asm:
            for l in asm1:
                if ">" in l:
                    contigs += 1

        if contigs > 1:
            print(p, file = oh1)
        print(p, count, sep="\t", file=oh2)

In [16]:
os.mkdir("scripts")

In [20]:
!head ./data/phages.txt
!scp ./scripts/assembly1_contigs.py jbrown@eofe4.mit.edu:~/scripts/

1.003.O.
1.004.O.
1.005.O.
1.007.O.
1.008.O.
1.009.O.
1.011.O.
1.012.O.
1.013.O.
1.015.O.
assembly1_contigs.py                          100%  925     0.9KB/s   00:00    


Want to figure out how many of the initial assemblies have more than one contig associated with it.

In [ ]:
with open(phagelist) as ih, open("multicontig.txt", "w") as oh1, open("asm_contig_cts.txt", "w") as oh2:
    print("phage", "assembly1_contig_count", sep="\t", file=oh2)
    for phage in ih:
        p = phage.strip()
        asm1 = "/nobackup1/jbrown/nahant_phage_reads/big_pipeline/{p}step1.clc.fasta".format(p=p)
        contigs = 0
        if op.exists(asm1):
            with open(asm1) as asm:
                for l in asm:
                    if ">" in l:
                        print(l)
                        contigs += 1

            if contigs > 1:
                print(p, file = oh1)

            print(p, contigs, sep="\t", file=oh2)

In [22]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_reads/multicontig.txt ./data/

multicontig.txt                               100% 1179     1.2KB/s   00:00    


In [9]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_reads/asm_contig_cts.txt ./data/

asm_contig_cts.txt                            100% 3227     3.2KB/s   00:00    


In [4]:
beginning = '''#!/bin/bash
#SBATCH -n 16
#SBATCH -N 1
#SBATCH -o check_asm1_bedcov.out
#SBATCH -e check_asm_bedcov.err
#SBATCH -p sched_mit_chisholm

module add engaging/clc-assembly-cell
module add engaging/samtools/1.1
module add engaging/bedtools/2.22.1
'''

phagelist = "./data/multicontig.txt"



Write slurm submission for read recruitment on multi-contig assemblies.

In [3]:
from __future__ import print_function

with open("./data/multicontig.txt") as ih, open("multicheck.slurm","w") as oh:
    print(beginning, file=oh)
    for p in ih:
        phage=p.strip()

        pdir = "/nobackup1/jbrown/nahant_phage_reads/"
        wdir = "multi_contig_check"
        cdir = "big_pipeline"
        fastq = op.join(pdir,"{phage}_*.fastq".format(**locals()))
        contig_assembly = op.join(pdir, cdir, "{phage}step1.clc.fasta".format(**locals()))
        outcas = op.join(pdir, wdir, "{phage}step1.clc.cas".format(**locals()))
        outbam = op.join(pdir, wdir, "{phage}step1.clc.bam".format(**locals()))
        rearranged = op.join(pdir, wdir, "{phage}step1.clc.sorted".format(**locals()))
        genomecov = op.join(pdir, wdir, "{phage}step1.clc.genomecoverage".format(**locals()))

        clc_map_cmd = "clc_mapper -o {outcas} -p fb ss 100 300 -q {fastq} -d {contig_assembly} -a local -r ignore".format(**locals())
        clc_to_sam = "clc_cas_to_sam -a {outcas} -o {outbam}".format(**locals())
        sam_sort = "samtools sort {outbam} {rearranged}".format(**locals())
        sam_index = "samtools index {rearranged}".format(**locals())
        bed_gcov = "bedtools genomecov -ibam {rearranged} -d > {genomecov}".format(**locals())
        
        print(clc_map_cmd, clc_to_sam, sam_sort, sam_index, bed_gcov, sep="\n", file=oh)
        print("\n", file=oh)


IOError: [Errno 2] No such file or directory: 'phages.txt'

In [39]:
!open multicheck.slurm -a textedit

In [40]:
!scp multicheck.slurm jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_reads/

multicheck.slurm                              100%  117KB 116.7KB/s   00:00    


In [6]:
from __future__ import print_function
import os.path as op

with open("./data/multicontig.txt") as ih, open("multibed.slurm","w") as oh:
    print(beginning, file=oh)
    for p in ih:
        phage=p.strip()
        pdir = "/nobackup1/jbrown/nahant_phage_reads/"
        wdir = "multi_contig_check"
        cdir = "big_pipeline"
        rearranged = op.join(pdir, wdir, "{phage}step1.clc.sorted.bam".format(**locals()))
        genomecov = op.join(pdir, wdir, "{phage}step1.clc.genomecoverage".format(**locals()))
        
        bed_gcov = "bedtools genomecov -ibam {rearranged} -d > {genomecov}".format(**locals())
        
        print(bed_gcov, file=oh)
        print("\n", file=oh)

In [7]:
!open multibed.slurm -a textedit

In [8]:
!scp multibed.slurm jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_reads/

multibed.slurm                                100%   27KB  26.6KB/s   00:00    


In [10]:
with open("./data/asm_contig_cts.txt") as ih:
    for l in ih:
        print(l)

phage	assembly1_contig_count

1.003.O.	1

1.004.O.	61

1.005.O.	67

1.007.O.	27

1.008.O.	1

1.009.O.	44

1.011.O.	5

1.012.O.	26

1.013.O.	93

1.015.O.	1

1.016.O.	62

1.017.O.	36

1.020.O.	223

1.021.A.	7

1.021.B.	6

1.021.C.	5

1.022.O.	2

1.023.O.	5

1.024.O.	80

1.025.O.	1

1.026.O.	1

1.027.O.	1

1.028.O.	98

1.029.O.	43

1.030.O.	1

1.031.O.	85

1.032.O.	236

1.033.O.	21

1.034.O.	1

1.034.X.	27

1.036.O.	15

1.037.O.	6

1.038.O.	1

1.039.O.	42

1.040.O.	32

1.042.O.	2

1.043.O.	57

1.044.O.	2

1.046.O.	4

1.047.O.	19

1.048.O.	8

1.049.O.	1

1.050.O.	14

1.052.A.	40

1.054.O.	1

1.055.O.	1

1.056.O.	6

1.057.O.	1

1.060.A.	1

1.061.O.	3

1.062.O.	19

1.063.O.	12

1.064.O.	1

1.066.O.	3

1.067.O.	57

1.068.O.	34

1.069.O.	7

1.070.O.	1

1.071.A.	1

1.072.O.	11

1.074.O.	1

1.075.O.	4

1.076.O.	1

1.077.O.	6

1.079.O.	1

1.080.O.	1

1.081.O.	1

1.082.O.	1

1.083.O.	1

1.084.O.	1

1.085.O.	1

1.086.O.	12

1.087.A.	3

1.088.O.	10

1.089.O.	1

1.090.B.	1

1.091.O.	1

1.093.O.	1

1.